In [ ]:
"""
Part 1: Multi-Armed Bandit Algorithms for Profit-Aware Product Recommendation
This module implements various MAB strategies for e-commerce product recommendations.

Author: DRL Assignment Solution
Date: December 2025
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple, Dict
import warnings
warnings.filterwarnings('ignore')


class MultiArmedBandit:
    """
    Multi-Armed Bandit base class for product recommendation.
    Handles the environment with 6 products (arms) and user sessions.
    """

    def __init__(self, data: pd.DataFrame, n_arms: int = 6):
        """
        Initialize the MAB environment.

        Args:
            data: DataFrame containing user sessions with product revenues and costs
            n_arms: Number of products/arms (default: 6)
        """
        self.data = data
        self.n_arms = n_arms
        self.n_users = len(data)
        self.current_user = 0
        self.net_rewards = self._compute_net_rewards()

    def _compute_net_rewards(self) -> np.ndarray:
        """
        Compute net rewards (profit) for all products across all users.
        NetReward = Revenue - Cost

        Returns:
            Array of shape (n_users, n_arms) with net rewards
        """
        net_rewards = np.zeros((self.n_users, self.n_arms))

        for arm in range(self.n_arms):
            product_col = f'Product{arm + 1}'
            cost_col = f'cost{arm + 1}'
            net_rewards[:, arm] = self.data[product_col].values - self.data[cost_col].values

        return net_rewards

    def reset(self):
        """Reset the environment to start a new simulation."""
        self.current_user = 0

    def step(self, action: int) -> Tuple[float, bool]:
        """
        Take an action (select a product) and observe the reward.

        Args:
            action: Product index (0-5)

        Returns:
            Tuple of (reward, done)
        """
        if self.current_user >= self.n_users:
            return 0.0, True

        reward = self.net_rewards[self.current_user, action]
        self.current_user += 1
        done = self.current_user >= self.n_users

        return reward, done

    def get_optimal_arm(self, user_idx: int) -> int:
        """
        Get the optimal arm for a specific user.

        Args:
            user_idx: User index

        Returns:
            Arm index with highest reward
        """
        return np.argmax(self.net_rewards[user_idx])


class RandomPolicy:
    """
    Random recommendation policy: randomly select one product for each user.
    This represents the current organizational policy.
    """

    def __init__(self, n_arms: int):
        """
        Initialize random policy.

        Args:
            n_arms: Number of products/arms
        """
        self.n_arms = n_arms

    def select_action(self) -> int:
        """
        Select a random action.

        Returns:
            Random arm index
        """
        return np.random.randint(0, self.n_arms)

    def update(self, action: int, reward: float):
        """Update policy (no learning in random policy)."""
        pass


class GreedyPolicy:
    """
    Greedy policy: try each product a few times, then always select the best one.
    This can lead to premature lock-in on a suboptimal product.
    """

    def __init__(self, n_arms: int, exploration_rounds: int = 50):
        """
        Initialize greedy policy.

        Args:
            n_arms: Number of products/arms
            exploration_rounds: Number of initial rounds to explore each arm
        """
        self.n_arms = n_arms
        self.exploration_rounds = exploration_rounds
        self.counts = np.zeros(n_arms)
        self.values = np.zeros(n_arms)
        self.t = 0

    def select_action(self) -> int:
        """
        Select action using greedy strategy.

        Returns:
            Arm index
        """
        self.t += 1

        if self.t <= self.exploration_rounds:
            return (self.t - 1) % self.n_arms
        else:
            return np.argmax(self.values)

    def update(self, action: int, reward: float):
        """
        Update value estimates.

        Args:
            action: Arm that was selected
            reward: Observed reward
        """
        self.counts[action] += 1
        n = self.counts[action]
        value = self.values[action]
        self.values[action] = value + (reward - value) / n


class EpsilonGreedyPolicy:
    """
    Epsilon-Greedy policy: mostly exploit the best product, but occasionally explore others.
    This balances exploration and exploitation.
    """

    def __init__(self, n_arms: int, epsilon: float = 0.1):
        """
        Initialize epsilon-greedy policy.

        Args:
            n_arms: Number of products/arms
            epsilon: Exploration probability (0-1)
        """
        self.n_arms = n_arms
        self.epsilon = epsilon
        self.counts = np.zeros(n_arms)
        self.values = np.zeros(n_arms)

    def select_action(self) -> int:
        """
        Select action using epsilon-greedy strategy.

        Returns:
            Arm index
        """
        if np.random.random() < self.epsilon:
            return np.random.randint(0, self.n_arms)
        else:
            return np.argmax(self.values)

    def update(self, action: int, reward: float):
        """
        Update value estimates.

        Args:
            action: Arm that was selected
            reward: Observed reward
        """
        self.counts[action] += 1
        n = self.counts[action]
        value = self.values[action]
        self.values[action] = value + (reward - value) / n


class UCBPolicy:
    """
    Upper Confidence Bound (UCB) policy: explore products with uncertain performance.
    This addresses the exploration-exploitation tradeoff more intelligently.
    """

    def __init__(self, n_arms: int, c: float = 2.0):
        """
        Initialize UCB policy.

        Args:
            n_arms: Number of products/arms
            c: Exploration parameter
        """
        self.n_arms = n_arms
        self.c = c
        self.counts = np.zeros(n_arms)
        self.values = np.zeros(n_arms)
        self.t = 0

    def select_action(self) -> int:
        """
        Select action using UCB strategy.

        Returns:
            Arm index
        """
        self.t += 1

        for arm in range(self.n_arms):
            if self.counts[arm] == 0:
                return arm

        ucb_values = np.zeros(self.n_arms)
        for arm in range(self.n_arms):
            bonus = self.c * np.sqrt(np.log(self.t) / self.counts[arm])
            ucb_values[arm] = self.values[arm] + bonus

        return np.argmax(ucb_values)

    def update(self, action: int, reward: float):
        """
        Update value estimates.

        Args:
            action: Arm that was selected
            reward: Observed reward
        """
        self.counts[action] += 1
        n = self.counts[action]
        value = self.values[action]
        self.values[action] = value + (reward - value) / n


def run_simulation(env: MultiArmedBandit, policy, n_rounds: int) -> Dict:
    """
    Run a simulation with a given policy.

    Args:
        env: MAB environment
        policy: Policy to use
        n_rounds: Number of rounds to simulate

    Returns:
        Dictionary with simulation results
    """
    env.reset()
    rewards = []
    actions = []
    cumulative_rewards = []
    total_reward = 0

    for _ in range(n_rounds):
        action = policy.select_action()
        reward, done = env.step(action)

        policy.update(action, reward)

        rewards.append(reward)
        actions.append(action)
        total_reward += reward
        cumulative_rewards.append(total_reward)

        if done:
            env.reset()

    return {
        'rewards': np.array(rewards),
        'actions': np.array(actions),
        'cumulative_rewards': np.array(cumulative_rewards),
        'total_reward': total_reward,
        'average_reward': total_reward / n_rounds,
        'action_counts': np.bincount(actions, minlength=env.n_arms)
    }


def analyze_net_rewards(net_rewards: np.ndarray) -> Dict:
    """
    Analyze net rewards to identify best and worst products.

    Args:
        net_rewards: Array of net rewards (n_users, n_arms)

    Returns:
        Dictionary with analysis results
    """
    mean_rewards = net_rewards.mean(axis=0)
    median_rewards = np.median(net_rewards, axis=0)
    positive_counts = (net_rewards > 0).sum(axis=0)

    best_product = np.argmax(mean_rewards)
    worst_product = np.argmin(mean_rewards)

    return {
        'mean_rewards': mean_rewards,
        'median_rewards': median_rewards,
        'positive_counts': positive_counts,
        'best_product': best_product + 1,
        'worst_product': worst_product + 1,
        'best_mean_reward': mean_rewards[best_product],
        'worst_mean_reward': mean_rewards[worst_product]
    }


def plot_cumulative_rewards(results_dict: Dict[str, Dict], title: str = "Cumulative Net Profit Comparison"):
    """
    Plot cumulative rewards for multiple strategies.

    Args:
        results_dict: Dictionary mapping strategy names to results
        title: Plot title
    """
    plt.figure(figsize=(12, 6))

    for strategy_name, results in results_dict.items():
        plt.plot(results['cumulative_rewards'], label=strategy_name, linewidth=2)

    plt.xlabel('Round', fontsize=12)
    plt.ylabel('Cumulative Net Profit', fontsize=12)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


def plot_reward_distribution(net_rewards: np.ndarray):
    """
    Plot distribution of net rewards for all products.

    Args:
        net_rewards: Array of net rewards (n_users, n_arms)
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    axes = axes.flatten()

    for i in range(6):
        axes[i].hist(net_rewards[:, i], bins=30, alpha=0.7, color=f'C{i}', edgecolor='black')
        axes[i].axvline(net_rewards[:, i].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
        axes[i].set_title(f'Product {i+1}', fontweight='bold')
        axes[i].set_xlabel('Net Reward')
        axes[i].set_ylabel('Frequency')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()


def main():
    """
    Main function to run all experiments and answer assignment questions.
    """
    print("="*80)
    print("PART 1: MULTI-ARMED BANDIT FOR PRODUCT RECOMMENDATION")
    print("="*80)
    print()

    url = "https://raw.githubusercontent.com/SahithiSiripuram/drl/main/Dataset_Product_Recommendation.csv"
    print(f"Loading dataset from: {url}")

    try:
        data = pd.read_csv(url)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
        print(f"Columns: {list(data.columns)}")
        print()
    except Exception as e:
        print(f"Error loading dataset: {e}")
        print("Creating synthetic dataset for demonstration...")
        n_users = 498
        data = pd.DataFrame()
        data['UserID'] = range(1, n_users + 1)

        np.random.seed(42)
        for i in range(1, 7):
            data[f'Product{i}'] = np.random.gamma(5, 10, n_users) + np.random.normal(0, 5, n_users)
            data[f'cost{i}'] = np.random.gamma(3, 5, n_users) + np.random.normal(0, 2, n_users)
        print("Synthetic dataset created!")
        print()

    env = MultiArmedBandit(data)

    print("\n" + "="*80)
    print("Q1: NET REWARD ANALYSIS")
    print("="*80)

    analysis = analyze_net_rewards(env.net_rewards)
    print(f"\nAverage Net Rewards by Product:")
    for i, mean_rew in enumerate(analysis['mean_rewards']):
        print(f"  Product {i+1}: ${mean_rew:.2f}")

    print(f"\nBest Product: Product {analysis['best_product']} (avg: ${analysis['best_mean_reward']:.2f})")
    print(f"Worst Product: Product {analysis['worst_product']} (avg: ${analysis['worst_mean_reward']:.2f})")

    print(f"\nPositive Profit Sessions by Product:")
    for i, count in enumerate(analysis['positive_counts']):
        pct = (count / env.n_users) * 100
        print(f"  Product {i+1}: {count}/{env.n_users} ({pct:.1f}%)")

    plot_reward_distribution(env.net_rewards)

    print("\n" + "="*80)
    print("Q2: RANDOM POLICY SIMULATION")
    print("="*80)

    n_rounds = 500
    print(f"\nSimulating {n_rounds} rounds with Random Policy...")

    random_policy = RandomPolicy(env.n_arms)
    random_results = run_simulation(env, random_policy, n_rounds)

    print(f"\nResults:")
    print(f"  Total Profit: ${random_results['total_reward']:.2f}")
    print(f"  Average Profit per Round: ${random_results['average_reward']:.2f}")
    print(f"\n  Product Selection Counts:")
    for i, count in enumerate(random_results['action_counts']):
        pct = (count / n_rounds) * 100
        print(f"    Product {i+1}: {count} times ({pct:.1f}%)")

    print("\n" + "="*80)
    print("Q3: GREEDY POLICY SIMULATION")
    print("="*80)

    print(f"\nSimulating {n_rounds} rounds with Greedy Policy...")
    print("Strategy: Try each product a few times, then always pick the best")

    greedy_policy = GreedyPolicy(env.n_arms, exploration_rounds=60)
    greedy_results = run_simulation(env, greedy_policy, n_rounds)

    print(f"\nResults:")
    print(f"  Total Profit: ${greedy_results['total_reward']:.2f}")
    print(f"  Average Profit per Round: ${greedy_results['average_reward']:.2f}")
    print(f"\n  Product Selection Counts:")
    for i, count in enumerate(greedy_results['action_counts']):
        pct = (count / n_rounds) * 100
        print(f"    Product {i+1}: {count} times ({pct:.1f}%)")

    most_chosen = np.argmax(greedy_results['action_counts'])
    print(f"\n  Most Frequently Chosen: Product {most_chosen + 1}")

    print("\n" + "="*80)
    print("Q4: EPSILON-GREEDY WITH DIFFERENT EXPLORATION RATES")
    print("="*80)

    exploration_rates = [0.02, 0.10, 0.25]
    epsilon_results = {}

    for epsilon in exploration_rates:
        print(f"\nSimulating with {epsilon*100:.0f}% exploration rate...")
        policy = EpsilonGreedyPolicy(env.n_arms, epsilon=epsilon)
        results = run_simulation(env, policy, n_rounds)
        epsilon_results[f'ε={epsilon*100:.0f}%'] = results

        print(f"  Total Profit: ${results['total_reward']:.2f}")
        print(f"  Average Profit per Round: ${results['average_reward']:.2f}")
        print(f"  Product Selection Counts:")
        for i, count in enumerate(results['action_counts']):
            pct = (count / n_rounds) * 100
            print(f"    Product {i+1}: {count} times ({pct:.1f}%)")

    best_epsilon = max(epsilon_results.items(), key=lambda x: x[1]['total_reward'])
    print(f"\nBest Exploration Rate: {best_epsilon[0]} with total profit ${best_epsilon[1]['total_reward']:.2f}")

    print("\n" + "="*80)
    print("Q5: UCB POLICY - INTELLIGENT EXPLORATION")
    print("="*80)

    print(f"\nSimulating {n_rounds} rounds with UCB Policy...")
    print("Strategy: Explore products with uncertain performance")

    ucb_policy = UCBPolicy(env.n_arms, c=2.0)
    ucb_results = run_simulation(env, ucb_policy, n_rounds)

    print(f"\nResults:")
    print(f"  Total Profit: ${ucb_results['total_reward']:.2f}")
    print(f"  Average Profit per Round: ${ucb_results['average_reward']:.2f}")
    print(f"\n  Product Selection Counts:")
    for i, count in enumerate(ucb_results['action_counts']):
        pct = (count / n_rounds) * 100
        print(f"    Product {i+1}: {count} times ({pct:.1f}%)")

    min_trials = np.argmin(ucb_results['action_counts'])
    print(f"\n  Product with Minimal Trials: Product {min_trials + 1} ({ucb_results['action_counts'][min_trials]} times)")
    print("  This product was tried initially but received minimal trials later,")
    print("  indicating UCB quickly learned it was suboptimal.")

    print("\n" + "="*80)
    print("Q6: COMPREHENSIVE COMPARISON")
    print("="*80)

    all_results = {
        'Random': random_results,
        'Greedy': greedy_results,
        'ε-Greedy (2%)': epsilon_results['ε=2%'],
        'ε-Greedy (10%)': epsilon_results['ε=10%'],
        'ε-Greedy (25%)': epsilon_results['ε=25%'],
        'UCB': ucb_results
    }

    print("\nStrategy Performance Summary:")
    print("-" * 80)
    print(f"{'Strategy':<20} {'Total Profit':>15} {'Avg Profit/Round':>20} {'Best Product':>15}")
    print("-" * 80)

    for name, results in all_results.items():
        best_product = np.argmax(results['action_counts']) + 1
        print(f"{name:<20} ${results['total_reward']:>14,.2f} ${results['average_reward']:>18,.2f} {'Product ' + str(best_product):>15}")

    print("-" * 80)

    best_strategy = max(all_results.items(), key=lambda x: x[1]['total_reward'])
    print(f"\nBest Strategy: {best_strategy[0]}")
    print(f"Total Profit: ${best_strategy[1]['total_reward']:.2f}")
    print(f"Average Profit per Round: ${best_strategy[1]['average_reward']:.2f}")

    most_profitable_product = np.argmax(analysis['mean_rewards']) + 1
    print(f"\nMost Consistently Profitable Product: Product {most_profitable_product}")
    print(f"Average Net Reward: ${analysis['mean_rewards'][most_profitable_product-1]:.2f}")

    print("\n" + "="*80)
    print("PLOTTING CUMULATIVE PROFIT CURVES")
    print("="*80)

    plot_cumulative_rewards(all_results, "Cumulative Net Profit: All Strategies")

    print("\n" + "="*80)
    print("CONCLUSIONS")
    print("="*80)
    print("""
1. The Random policy provides unstable and suboptimal performance, serving as a baseline.

2. The Greedy policy can get stuck with suboptimal products due to premature lock-in.

3. Epsilon-Greedy policies balance exploration and exploitation:
   - Low exploration (2%): May miss better options
   - Moderate exploration (10%): Good balance for this dataset
   - High exploration (25%): Too much exploration reduces profit

4. UCB intelligently explores uncertain products, often achieving the best performance.

5. Learning-based approaches significantly outperform the random policy.

RECOMMENDATION: Implement UCB or ε-Greedy (10%) for production deployment.
    """)


if __name__ == "__main__":
    main()
